In [1]:
import sys
import random
from pathlib import Path
import torch

import networkx as nx

# Make sure we can import the dagc package from src/
ROOT = Path("/Users/nicholaskhorasani/Documents/CodingProjects/diffusion-aware-graph-compression")
sys.path.append(str(ROOT / "src"))

from dagc.graphs import read_graph, generate_seed_set


# -----------------------------
# Experiment configuration
# -----------------------------
graph_path = ROOT / "data" / "graphs" / "ba_1000_3.edgelist"

num_runs = 100
seed_set_size = 5
activation_prob = 0.1
keep_ratio = 0.5  # keep 50% of edges → remove 50%
base_seed = 12345

# Path to a trained GNN checkpoint (to be produced by your training script)
model_checkpoint = ROOT / "checkpoints" / "gnn_sparsifier.pt"
device = "cpu"  # or "cuda" if available

print("=== GNN Edge Sparsification Experiment ===")
print(f"Graph path:       {graph_path}")
print(f"Num runs:         {num_runs}")
print(f"Seed set size:    {seed_set_size}")
print(f"Activation prob:  {activation_prob}")
print(f"Keep ratio:       {keep_ratio}")
print(f"Base RNG seed:    {base_seed}")
print(f"Model checkpoint: {model_checkpoint}")
print(f"Device:           {device}")
print()

# -----------------------------
# Load original graph
# -----------------------------
print("Loading original graph...")
G = read_graph(str(graph_path), fmt="edge_list")

ModuleNotFoundError: No module named 'dagc.graphs'

In [ ]:
nodes = sorted(G.nodes())              # e.g. [0,1,2,...] but could be arbitrary
node_to_idx = {u: i for i, u in enumerate(nodes)}
n = len(nodes)

src = []
dst = []
for u, v in G.edges():
    i = node_to_idx[u]
    j = node_to_idx[v]
    # if undirected, we store both directions for message passing
    src.append(i); dst.append(j)
    src.append(j); dst.append(i)

edge_index = torch.tensor([src, dst], dtype=torch.long)  # shape: [2, num_edges_dir]


In [ ]:
edge_index

tensor([[  0,   1,   0,  ..., 885, 896, 937],
        [  1,   0,   2,  ..., 913, 937, 896]])

In [ ]:
import torch

degrees = torch.tensor(
    [G.degree[u] for u in nodes],
    dtype=torch.float32,
).unsqueeze(-1)  # [n, 1]

ones = torch.ones_like(degrees)        # [n, 1]

x = torch.cat([degrees, ones], dim=1)  # [n, 2] → in_dim = 2


In [ ]:
degrees.shape

torch.Size([1000, 1])

In [ ]:
x

tensor([[52.,  1.],
        [31.,  1.],
        [ 4.,  1.],
        ...,
        [ 3.,  1.],
        [ 3.,  1.],
        [ 3.,  1.]])

# 🧠 Understanding the Forward Pass of `SimpleGCNLayer`

These are my notes on *exactly* what happens inside one GCN layer — how node features propagate, what `x_src` is doing, how `index_add_` works, what shapes everything has, and how the learnable weights get applied.  
This is written in my own tone so I fully understand every step.

---

# 1. Objects and Shapes

We start with:

- **x**: node feature matrix  
  Shape: `[num_nodes, in_dim]`  
  Example: `[1000, 2]` → 1000 nodes with 2 features (degree + constant 1)

- **edge_index**  
  Shape: `[2, num_edges_dir]`  
  Each column is `(src, dst)` for a directed edge.

Because the graph is undirected, we manually store **both directions**:
```
(u → v), (v → u)
```

- **src**: `[num_edges_dir]`  
- **dst**: `[num_edges_dir]`

Our GCN layer implements:
```
h_new(v) = ReLU( W_self * h(v)  +  W_neigh * mean_{u in N(v)} h(u) )
```

Where:
- `h(v)` is the current node feature
- `N(v)` are v’s neighbors
- `W_self`, `W_neigh` are shared learnable matrices

---

# 2. What Are h(v), h(u), and N(v)?

- **h(v)** — the feature vector for node v at the current layer (initially `x[v]`)
- **h(u)** — same thing, but for neighbor u
- **N(v)** — neighbors of v

Important:
> Each node **does NOT have its own weights**.  
> All nodes use the *same* W_self and W_neigh.

This is what makes GNNs generalize.

---

# 3. Computing `x_src` — messages from source nodes

Given:
```
src = [s0, s1, s2, ..., sK]
```

We gather source-node features:

```
x_src = x[src]          # shape: [num_edges_dir, in_dim]
```

Meaning:
- `x_src[i] = x[src[i]]`
- If edge i is `0 → 1`, `x_src[i] = x[0]`.

So:
> `x_src` is the “message” attached to every edge, copied from the node where the edge *starts*.

---

# 4. Understanding `index_add_`: summing messages into destination nodes

We want:
```
neigh_agg[v] = sum of all x_src[i] where dst[i] = v
```

Initialize:
```
neigh_agg = zeros_like(x)     # shape: [num_nodes, in_dim]
```

Then:
```
neigh_agg.index_add_(0, dst, x_src)
```

This means:

For each edge i:
```
node = dst[i]
neigh_agg[node] += x_src[i]
```

So:
> neigh_agg[v] becomes the sum of all incoming neighbor feature vectors to v.

This matches exactly what I expected:
- For node v, we add up all feature vectors of nodes that point to v.

---

# 5. Compute Degrees Using the Same Trick

We count how many edges go into each node:

```
deg = torch.zeros(num_nodes)
deg.index_add_(0, dst, torch.ones_like(dst))
```

This does:
```
deg[v] = number of incoming edges to v
```

Then:
```
deg = deg.clamp(min=1).unsqueeze(-1)
```
so we avoid dividing by zero.

---

# 6. Mean Neighbor Features

Divide neighbor sums by degree:

```
neigh_mean = neigh_agg / deg     # shape: [num_nodes, in_dim]
```

So:
> neigh_mean[v] is literally the average of all neighbor features of node v.

---

# 7. What Are W_self and W_neigh?

These are **matrices** created by:

```
self.W_self  = nn.Linear(in_dim, out_dim, bias=True)
self.W_neigh = nn.Linear(in_dim, out_dim, bias=False)
```

If:
- `in_dim = 2`
- `out_dim = 64`

Then:

- `W_self.weight`: `[64, 2]`  
- `W_self.bias`: `[64]`  
- `W_neigh.weight`: `[64, 2]`  

These are the parameters the model will **learn**.

---

# 8. What Happens Inside `W_self(x)`?

PyTorch performs:
```
W_self(x)  =  x @ W_self.weight^T  +  bias
```

Shapes:
- x: `[num_nodes, 2]`
- W_self.weight: `[64, 2]`
- result: `[num_nodes, 64]`

Same for:
```
W_neigh(neigh_mean)
```

Thus:
- each node’s own 2D vector becomes 64D
- each node’s neighbor-mean 2D vector also becomes 64D

---

# 9. Combining Everything

We compute:

```
out = W_self(x) + W_neigh(neigh_mean)
out = ReLU(out)
```

So for **each** node v:

```
h_self  = W_self * h(v)
h_neigh = W_neigh * mean_neighbor_features(v)
h_new(v) = ReLU( h_self + h_neigh )
```

The result:
- shape: `[num_nodes, out_dim]`  
- typically `[num_nodes, 64]`

This becomes the input to the next GCN layer.

---

# 10. Final Summary (my own words)

- Each node starts with simple features (like degree + 1).
- We copy each source-node's feature onto every outgoing edge → `x_src`.
- We sum these for each destination node using `index_add_` → `neigh_agg`.
- We divide by degree → `neigh_mean`.
- We apply **shared** learnable matrices:
  - `W_self` on the node’s own feature,
  - `W_neigh` on the mean neighbor feature.
- We add them and apply ReLU.
- This gives a new embedding per node (e.g. 64D), which feeds into the next layer.

These parameters update during training so that edge scores (produced later by an MLP) lead to a compressed graph that preserves diffusion behavior.

This is the entire forward pass, line by line, and now I understand every piece of it.


In [ ]:
from dataclasses import dataclass
from typing import Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleGCNLayer(nn.Module):
    """This is a very simple GCN-like Layer. Graph convolutional neural network
    h_new(v) = ReLU(W-self h(v) + W_neigh * mean_{u in N(v)} h(u))
    """
    def __init__(self, in_dim: int, out_dim: int):
        super().__init__()
        self.W_self = nn.Linear(in_dim, out_dim, bias = True)
        self.W_neigh = nn.Linear(in_dim, out_dim, bias = False)
    
    def forward(self, x: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        """
        x : [num_nodes, in_dim]
        edge_index: [2, num_edges_dir] with (src, dst) per column
        """
        num_nodes = x.size(0)
        src, dst = edge_index
        
        # Gather source node embeddings for each edge
        # Basically this is the "node features" of the "source node" of each edge. 
        # So the 0th element of x_src will be the node features of the source node of the 0th edge
        # this is probably the node features of node 0!
        x_src = x[src] #[num_edges_dir, in_dim]
        
        # Initialize neighbour aggregation with zeros
        neigh_agg = torch.zeros_like(x) # [num_nodes, in_dim]
        # neigh_agg[v] will store the sum of features received from all neighbors of v.
        # Aggregate (sum) messages into destination nodes
        neigh_agg.index_add_(0, dst, x_src) # Sum messages into neigh_add[dst]
        
        # Compute degree for normalization (numbwer of incoming edges)
        deg = torch.zeros(num_nodes, device = x.device, dtype = x.dtype)
        one_vec = torch.ones_like(src, dtype = x.dtype)
        deg.index_add_(0, dst, one_vec)
        # Avoid division by zero
        deg = torch.clamp(deg, min= 1.0).unsqueeze(-1) # [num_nodes, 1]
        
        neigh_mean = neigh_agg / deg # Mean Neighbor features
        
        out = self.W_self(x) + self.W_neigh(neigh_mean)
        return F.relu(out)
network = SimpleGCNLayer(2, 2)
network.forward(x, edge_index)

NameError: name 'x' is not defined

In [ ]:
class GNNEncoder(nn.Module):
    """
    Stack of simple GCNLayers
    h^(0) = x
    h^(l+1) = SimpleGCNLayer_l(h^(l), edge_index)
    """
    def __init__(self, in_dim: int, hidden_dim: int, num_layers: int):
        super().__init__()
        layers = []
        
        # First layer: in_dim -> hidden_dim
        layers.append(SimpleGCNLayer(in_dim, hidden_dim))
        
        #Remaining layers hidden_dim -> hidden_dim
        for _ in range(num_layers - 1):
            layers.append(SimpleGCNLayer(hidden_dim, hidden_dim))
        
        self.layers = nn.ModuleList(layers)
        
    def forward(self, x: torch.Tensor, edge_index: torch.Tensor) -> torch.Tensor:
        """
        x: [num_nodes, in_dim] (in_dim is the dimension of the features for node i)
        edge_index: [2, num_edges_dir]
        returns: H [num_nodes, hidden_dim] 
        """
        h = x
        for layer in self.layers:
            h = layer(h, edge_index)
        return h
        

In [ ]:
encoder = GNNEncoder(in_dim=2, hidden_dim=4, num_layers=2)
H = encoder(x, edge_index)
print(H.shape)  # should be [num_nodes, 4]

torch.Size([1000, 4])


In [ ]:
from dataclasses import dataclass
from typing import Optional, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F


@dataclass
class GNNSparsifierOutput:
    node_embeddings: torch.Tensor   # [num_nodes, hidden_dim]
    edge_logits: torch.Tensor       # [num_edges_dir]
    edge_probs: torch.Tensor        # [num_edges_dir]


class GNNSparsifier(nn.Module):
    def __init__(
        self, 
        in_dim: int, 
        hidden_dim: int = 64,
        num_layers: int = 2,
        edge_mlp_hidden_dim: int = 64,
    ):
        super().__init__()
        self.encoder = GNNEncoder(
            in_dim=in_dim,
            hidden_dim=hidden_dim,
            num_layers=num_layers,
        )
        
        # Edge feature dimension: [h_u, h_v, h_u * h_v, |h_u - h_v|]
        edge_feat_dim = 4 * hidden_dim
        
        self.edge_mlp = nn.Sequential(
            nn.Linear(edge_feat_dim, edge_mlp_hidden_dim),
            nn.ReLU(), 
            nn.Linear(edge_mlp_hidden_dim, 1),  # scalar logit per edge
        )

    def forward(
        self, 
        x: torch.Tensor,          # [num_nodes, in_dim]
        edge_index: torch.Tensor, # [2, num_edges_dir]
    ) -> GNNSparsifierOutput:
        # Step 1) Node embeddings from GNN
        H = self.encoder(x, edge_index)  # [num_nodes, hidden_dim]
        
        # Step 2) Build per-edge features 
        src, dst = edge_index
        
        # Learned embedding for the source / dest node of each edge
        h_src = H[src]  # [num_edges_dir, hidden_dim]
        h_dst = H[dst]  # [num_edges_dir, hidden_dim]
        
        edge_feat = torch.cat(
            [h_src, h_dst, h_src * h_dst, torch.abs(h_src - h_dst)],
            dim=-1,
        )  # [num_edges_dir, 4 * hidden_dim]
        
        # Step 3) Edge logits + probabilities
        edge_logits = self.edge_mlp(edge_feat).squeeze(-1)  # [num_edges_dir]
        edge_probs = torch.sigmoid(edge_logits)             # [num_edges_dir]
        
        return GNNSparsifierOutput(
            node_embeddings=H,
            edge_logits=edge_logits,
            edge_probs=edge_probs,
        )


In [ ]:
def graph_to_tensors(G: nx.Graph, device: Optional[torch.device] = None) -> tuple[torch.Tensor, torch.Tensor]:
    